# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [28]:
print(get_links_user_prompt("https://www.ibm.com/services/penetration-testing"))


Here is the list of links on the website https://www.ibm.com/services/penetration-testing -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://www.ibm.com/reports/data-breach
/
/services
https://www.ibm.com/reports/threat-intelligence
https://www.ibm.com/account/reg/signup?formid=urx-52262
https://www.ibm.com/account/reg/signup?formid=urx-52262
https://www.ibm.com/think/x-force
https://www.ibm.com/new/announcements/x-force-cloud-threat-landscape
https://www.ibm.com/downloads/documents/us-en/107a02e951c8fd86
https://www.ibm.com/account/reg/signup?formid=news-urx-52954
https://www.ibm.com/think/x-force
https://www.ibm.com/reports/data-breach
https://www.ibm.com/reports/threat-intelligence
https://www.ibm.com/account/reg/signup?formid=urx-53217
https://www.ibm.com/think/x-force/remotemonologue-weaponizin

In [29]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links


In [30]:
select_relevant_links("https://www.ibm.com/services/penetration-testing")

{'links': [{'type': 'service page',
   'url': 'https://www.ibm.com/services/penetration-testing'},
  {'type': 'service page',
   'url': 'https://www.ibm.com/services/adversary-simulation'},
  {'type': 'service page',
   'url': 'https://www.ibm.com/services/offensive-security'},
  {'type': 'report', 'url': 'https://www.ibm.com/reports/data-breach'},
  {'type': 'report', 'url': 'https://www.ibm.com/reports/threat-intelligence'},
  {'type': 'brand page', 'url': 'https://www.ibm.com/think/x-force'},
  {'type': 'brand page', 'url': 'https://www.ibm.com/x-force'},
  {'type': 'news',
   'url': 'https://www.ibm.com/new/announcements/x-force-cloud-threat-landscape'},
  {'type': 'document',
   'url': 'https://www.ibm.com/downloads/documents/us-en/107a02e951c8fd86'},
  {'type': 'thought leadership article',
   'url': 'https://www.ibm.com/think/x-force/remotemonologue-weaponizing-dcom-ntlm-authentication-coercions'},
  {'type': 'thought leadership article',
   'url': 'https://www.ibm.com/think/x-f

In [31]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [32]:
select_relevant_links("https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'services page', 'url': 'https://www.ibm.com/services'},
  {'type': 'security research / X-Force',
   'url': 'https://www.ibm.com/x-force'},
  {'type': 'adversary simulation service',
   'url': 'https://www.ibm.com/services/adversary-simulation'},
  {'type': 'offensive security service',
   'url': 'https://www.ibm.com/services/offensive-security'},
  {'type': 'X-Force thought leadership',
   'url': 'https://www.ibm.com/think/x-force'},
  {'type': 'careers page', 'url': 'https://ibmglobal.avature.net/globalform'}]}

In [33]:
select_relevant_links("https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'services overview',
   'url': 'https://www.ibm.com/services'},
  {'type': 'services page',
   'url': 'https://www.ibm.com/services/adversary-simulation'},
  {'type': 'services page',
   'url': 'https://www.ibm.com/services/offensive-security'},
  {'type': 'press release',
   'url': 'https://www.ibm.com/new/announcements/x-force-cloud-threat-landscape'},
  {'type': 'brand page', 'url': 'https://www.ibm.com/x-force'},
  {'type': 'think leadership', 'url': 'https://www.ibm.com/think/x-force'},
  {'type': 'thought leadership',
   'url': 'https://www.ibm.com/reports/data-breach'},
  {'type': 'thought leadership',
   'url': 'https://www.ibm.com/reports/threat-intelligence'},
  {'type': 'careers page', 'url': 'https://ibmglobal.avature.net/globalform'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [23]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        #result += f"\n\n### Link: {link['type']}\n"
        result += f"\n\n### Link: {link['type']}\nURL: {link['url']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [34]:
print(fetch_page_and_all_relevant_links("https://www.ibm.com/services/penetration-testing"))

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 0 relevant links
## Landing Page:

Penetration Testing Services | IBM

Cost of a Data Breach Report 2025
AI is becoming a security risk. Are you covered?
Home
Services
Penetration Testing
X-Force Red Penetration Testing Services
Testing AI models, applications, networks, hardware, and personnel to uncover and fix vulnerabilities
Read the Threat Intelligence Index Report
Schedule a briefing
Identify and fix critical vulnerabilities across your enterprise
X-Force Red delivers penetration testing services for your applications, networks, cloud assets, AI models, mainframes, hardware, personnel and more to uncover vulnerabilities and misconfigurations that could lead to unauthorized access to systems or sensitive data. With decades of experience breaking into organizations using the same tools, techniques, practices, and mindsets as criminals, X-Force Red offers the skills, scale, and 

In [42]:
#brochure_system_prompt = """
#You are an assistant that analyzes the contents of several relevant pages from a company website
#and creates a short brochure about the company for prospective customers, investors and recruits.
#Respond in markdown without code blocks.
#Include details of company culture, customers and careers/jobs if you have the information.
#"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
 You are a an assistant that analyzes the contents of several relevant pages from a company website
 and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
 Respond in markdown without code blocks.
 Include details of company culture, customers and careers/jobs if you have the information.
 """


In [43]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [35]:
get_brochure_user_prompt("IBM X-Force Red", "https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 8 relevant links


"\nYou are looking at a company called: IBM X-Force Red\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nPenetration Testing Services | IBM\n\nCost of a Data Breach Report 2025\nAI is becoming a security risk. Are you covered?\nHome\nServices\nPenetration Testing\nX-Force Red Penetration Testing Services\nTesting AI models, applications, networks, hardware, and personnel to uncover and fix vulnerabilities\nRead the Threat Intelligence Index Report\nSchedule a briefing\nIdentify and fix critical vulnerabilities across your enterprise\nX-Force Red delivers penetration testing services for your applications, networks, cloud assets, AI models, mainframes, hardware, personnel and more to uncover vulnerabilities and misconfigurations that could lead to unauthorized access to systems or sensitive data. With decades of experience breaking into organizatio

In [44]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [36]:
create_brochure("IBM X-Force Red", "https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 14 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# IBM X-Force Red Brochure

---

## About IBM X-Force Red

IBM X-Force Red is a premier penetration testing and adversary simulation service provider, specializing in identifying and fixing critical vulnerabilities across applications, networks, cloud assets, AI models, mainframes, hardware, and personnel. Leveraging decades of experience and the same tools, techniques, and mindsets as malicious hackers, X-Force Red empowers organizations to safeguard their most sensitive systems and data.

---

## Core Services

- **Penetration Testing**  
  Comprehensive security assessments including manual penetration testing, secure code reviews, binary analysis, and vulnerability assessments across platforms — web, mobile, APIs, and thick clients.

- **AI Security Testing**  
  Specialized testing to uncover vulnerabilities in Foundation Models and Large Language Models (FM/LLMs), MLSecOps pipelines, AI Platforms, and Generative AI (GenAI) applications, addressing the evolving AI security landscape.

- **Cloud Security Testing**  
  Evaluation of cloud environments to detect critical misconfigurations, insecure DevOps practices such as shared secrets, and inadvertent exposure of sensitive data through object storage or privilege escalation pathways.

- **Adversary Simulation**  
  Simulating sophisticated cyberattacks to test, measure, and improve incident detection, risk response, and remediation capabilities, ensuring organizations can detect and mitigate breaches quickly and effectively.

---

## Why Choose X-Force Red?

- **Expertise with a Hacker’s Mindset:**  
  Our team thinks and operates like attackers, giving you a unique perspective on vulnerabilities criminals seek to exploit.

- **Comprehensive Coverage:**  
  From legacy mainframes to cutting-edge AI models, X-Force Red tests every layer of your enterprise infrastructure.

- **Research-driven Insights:**  
  Stay ahead with access to the latest threat intelligence and analytical reports, including the Cost of a Data Breach Report and the X-Force Cloud Threat Landscape Report.

- **Trusted Partner for Incident Readiness:**  
  Beyond finding vulnerabilities, we test your incident response programs to ensure your defenses respond effectively under real attack scenarios.

---

## Our Customers

IBM X-Force Red serves a diverse global customer base, including enterprises across industries such as finance, healthcare, government, technology, and manufacturing. Our clients rely on us to protect critical assets from increasingly sophisticated cyber threats and regulatory challenges related to data and AI security.

---

## Company Culture

IBM X-Force Red thrives on innovation, collaboration, and relentless curiosity. Our culture fosters:

- **Continuous Learning:** Staying ahead of cyber threats through ongoing research and sharing of knowledge.
- **Diversity and Inclusion:** A global team encouraged to bring diverse perspectives and unique skills to solve complex security problems.
- **Integrity and Trust:** Building confidence with clients through transparency, professionalism, and commitment to protecting their digital environments.
- **Agility:** Rapid adaptation to emerging threats and technologies, including AI and cloud security.

---

## Careers at IBM X-Force Red

Join IBM X-Force Red and be part of a cutting-edge cybersecurity team dedicated to making the digital world safer. We seek talented professionals passionate about security, ethical hacking, AI risk mitigation, and cloud security.

- **Roles Available:** Penetration Testers, AI Security Analysts, Cloud Security Specialists, Incident Response Experts, Security Researchers.
- **Growth Opportunities:** Work with industry-leading tools, contribute to high-impact research, and collaborate with top global experts.
- **Employee Development:** Continuous training, certifications, and access to IBM’s vast resources and AI innovations.

Explore current openings and start your career in advanced cybersecurity with IBM X-Force Red.

---

## Connect with IBM X-Force Red

- **Schedule a Briefing:** Engage with our experts to understand your vulnerabilities and protection strategies.
- **Research Hub:** Access the latest insights and threat intelligence reports curated by our world-class hackers and analysts.
- **Learn More:** Visit [IBM Services](https://www.ibm.com/services) and [X-Force Red Penetration Testing](https://www.ibm.com/services/adversary-simulation) pages for deeper details.

---

**Protect your enterprise with IBM X-Force Red — where cutting-edge security testing meets real-world hacker expertise.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [38]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("IBM X-Force Red", "https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 11 relevant links


# IBM X-Force Red Brochure

---

## About IBM X-Force Red

IBM X-Force Red is a world-class penetration testing and adversary simulation team dedicated to uncovering and fixing critical security vulnerabilities across your enterprise. Leveraging decades of experience hacking into organizations using the same tactics, techniques, and mindsets as malicious actors, X-Force Red helps you protect your data, systems, and people from cyber threats.

---

## Our Services

### Penetration Testing

- **Comprehensive Testing:** Applications, networks, cloud assets, AI models, mainframes, hardware, and personnel.
- **Vulnerability Discovery:** Identify hidden risks, misconfigurations, and weaknesses that could lead to unauthorized access or data breaches.
- **Manual Penetration Testing:** Hands-on expert testing to simulate real-world attacks.
- **Secure Code Review & Binary Analysis:** Ensuring software integrity and security at the development level.
- **Platform Coverage:** Web, mobile, APIs, thick-client applications.

### AI Security Testing

- **AI Models & Platforms:** Evaluate security across Foundation Models (FM), Large Language Models (LLMs), and Generative AI (GenAI) applications.
- **MLSecOps Pipelines:** Address vulnerabilities through the entire machine learning lifecycle.
- **AI Solution Expertise:** Focused on uncovering and remediating risks unique to AI systems.

### Cloud Security Testing

- **Cloud Asset Penetration:** Identify critical misconfigurations and attack paths.
- **DevOps Security:** Detect insecure DevOps practices such as exposed credentials or improperly configured object storage.
- **Privilege Escalation Risks:** Analyze and thwart unauthorized privilege adoption.

### Adversary Simulation

- **Simulated Sophisticated Attacks:** Test, measure, and improve your risk detection and incident response capabilities.
- **Incident Response Testing:** Ensure your defenses can detect, contain, and mitigate real breaches effectively.
- **Shadow AI Detection:** Combat hidden and undetected AI-powered security threats.

---

## Why Choose IBM X-Force Red?

- **Decades of Expertise:** Real-world experience matching the latest attacker tactics.
- **Comprehensive Coverage:** From AI to cloud, hardware to human factors.
- **Cutting-Edge Research:** Access to insights from the X-Force Red Research Hub and the Cloud Threat Landscape Report.
- **Global Scale:** Leverage IBM’s extensive resources and AI expertise for robust security solutions.

---

## Company Culture

At IBM X-Force Red, innovation and collaboration are key. The team is composed of skilled hackers, security researchers, and AI specialists passionate about safeguarding organizations. Their proactive and ethical hacking mindset drives continuous learning and adaption, ensuring customers stay ahead of emerging threats. Diversity and inclusion fuel creativity and problem-solving to tackle the most complex security challenges.

---

## Customers and Impact

IBM X-Force Red partners with enterprises across industries—from finance, healthcare, and retail to government and technology. They help organizations protect sensitive data, meet compliance requirements, and foster secure adoption of AI and cloud technologies, enabling confident business growth.

---

## Careers at IBM X-Force Red

Join a team of elite security professionals dedicated to pushing the boundaries of cybersecurity. Careers span penetration testers, AI security researchers, cloud security experts, and red team operators. IBM offers a dynamic, supportive environment that encourages continuous skill development, innovation, and collaboration on groundbreaking security challenges.

---

## Learn More & Contact

- **Schedule an X-Force briefing:** Engage with experts to tailor your security strategy.
- **Research Hub:** Stay informed with the latest threat intelligence and security analysis.
- **Stay Ahead:** Download the X-Force Cloud Threat Landscape Report 2024.

Visit the [IBM X-Force Red Penetration Testing Services](https://www.ibm.com/services) page for more information.

---

**Protect your enterprise with IBM X-Force Red—turning insights into action to secure your future.**

In [45]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("IBM X-Force Red", "https://www.ibm.com/services/penetration-testing")

Selecting relevant links for https://www.ibm.com/services/penetration-testing by calling gpt-5-nano
Found 7 relevant links


# IBM X-Force Red: The Cybersecurity Superheroes You Didn’t Know You Needed

---

## Who Are We?  
Welcome to IBM X-Force Red — the *Sherlock Holmes* of cybersecurity, but with more tech gadgets and less deerstalker hats. We’re IBM’s elite penetration testing team, skilled in the fine art of “hacking” your own systems **before** the bad guys do. With decades of experience—and possibly some nerves of steel—we dive into your applications, AI models, cloud platforms, hardware, and even people (yep, social engineering too) to sniff out vulnerabilities lurking in the shadows.

---

## Our Superpowers

- 🔍 **AI Testing:** We test your cutting-edge Foundation Models, Large Language Models, AI pipelines, and GenAI applications to ensure no sinister loophole escapes. Because Tyrion Lannister said it best, "A mind needs books as a sword needs a whetstone," and yours needs protection.

- 🛠️ **Application Testing:** Web, mobile, thick clients, APIs—if it talks to the internet, we’ll break it, fix it, and report back. Includes manual penetration testing, secure code review, and binary analysis.

- ☁️ **Cloud Security Testing:** Hunting misconfigurations and insecure DevOps secrets like exposed API keys or object storage blunders. Because cloud leaks are so last decade.

- 🌐 **External Network Testing:** Our hackers think like the enemy to uncover paths that lead to unauthorized access. We attack your network so the real attackers can’t.

---

## Why Us?  

We’re not just penetration testers; we’re the friendly neighborhood adversaries working for you. Using the same tools, tactics, and mindset as cybercriminals, we uncover the most dangerous gaps before malicious actors do—because prevention beats the headache of a data breach every time.

Plus, we’re backed by IBM's legendary resources and research, including the **X-Force Threat Intelligence Index** and the **Cloud Threat Landscape Report 2024**—think of it as our hacker’s diary, full of valuable insights.

---

## Who Loves Us?

- From **Fortune 500 giants** to innovators pushing AI boundaries, companies trust us to protect their digital crown jewels.  
- Sports legends like UFC and Scuderia Ferrari love IBM’s AI magic, and though X-Force Red works behind the scenes, we power the speed, security, and thrills you enjoy.

---

## Our Culture: Hacker Zest Meets IBM Best

- A no-nonsense environment where curiosity *and* coffee flow in equal measure.  
- Diverse teams of cybersecurity pros who love a good challenge—ethical hacking, thoughtful research, and geeky banter encouraged.  
- Continuous learning is a way of life, especially because technology (and hackers) evolve faster than your favorite streaming service updates.  
- Collaboration, integrity, and a shared mission to outsmart cybercrime with wit, skill, and a sprinkle of hacker humor.

---

## Careers: Ready to Join the Good Side?

If you’re a penetration tester, bug bounty hunter, AI security enthusiast, or just someone who thinks “Ctrl+Alt+Del” is part of a magical ritual, come join the X-Force Red ranks! Here you’ll:

- Work with bleeding-edge tech like AI and cloud security.  
- Hack with purpose—protecting organizations worldwide.  
- Collaborate with world-class experts (and occasionally prank each other with terminal commands).  
- Access IBM’s trove of knowledge, tools, and career development resources.

**Pro Tip:** Your skills might get tested—literally. But rest assured, you’ll be joining a community that celebrates the hacker spirit with the ethics of a superhero.

---

## Let’s Talk!

Worried about AI models turning rogue or your cloud configuration being a hacker’s playground? Schedule an X-Force Red briefing today to see how we turn your vulnerabilities into victories.

**Warning:** Side effects may include: peace of mind, boosted security posture, and the urge to call yourself a cybersecurity ninja.

---

**IBM X-Force Red**  
*The defenders of your digital fortress. Because every empire deserves a loyal red team.*  

---

*Discover more, protect better, and maybe even have some fun—visit [IBM X-Force Red](https://www.ibm.com/services/security/ibm-x-force-red) and join the cybersecurity revolution.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>